In [1]:
import modelseedpy
import cobrakbase
import cobra
import json

cobrakbase 0.2.7


In [2]:
kbase = cobrakbase.KBaseAPI()
ws = 'filipeliu:narrative_1623733614496'

In [3]:
#genome = kbase.get_from_ws('ecoli_genome', ws)
from cobrakbase.core.kbasegenomesgenome import KBaseGenome
with open('ecoli_genome.json', 'r') as fh:
    genome_json = json.load(fh)
    genome = KBaseGenome(genome_json, cobrakbase.kbase_object_info.KBaseObjectInfo('1-1'))

In [4]:
with open('ecoli_genome_rast_annotation.json', 'r') as fh:
    ecoli_genome_rast_annotation = json.load(fh)
for f in genome.features:
    if f.id in ecoli_genome_rast_annotation:
        for function in ecoli_genome_rast_annotation[f.id]:
            f.add_ontology_term('RAST', function)

In [5]:
template = kbase.get_from_ws('12998/10/6')

In [6]:
model_modelseed = modelseedpy.MSBuilder.build_metabolic_model('ecoli', genome, template, '0', True, False)

In [7]:
kbase.save_object('ecoli_modelseedpy_direct', 'filipeliu:narrative_1623733614496', 'KBaseFBA.FBAModel', model_modelseed)

[[17,
  'ecoli_modelseedpy_direct',
  'KBaseFBA.FBAModel-12.0',
  '2021-06-17T08:28:05+0000',
  11,
  'filipeliu',
  92991,
  'filipeliu:narrative_1623733614496',
  '5a637663c45030603ad35da1b5515db5',
  1569215,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'ecoli',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Source': 'ModelSEEDpy',
   'Number compounds': '1448',
   'Number reactions': '1561',
   'Name': 'ecoli'}]]

In [4]:
rast = modelseedpy.RastClient()

In [5]:
rast.annotate_genome(genome)

[{'execution_time': 1623837791.78075,
  'id': '0F06406E-CE8A-11EB-B500-6F3DBCF382BD',
  'tool_name': 'kmer_search',
  'parameters': ['-a',
   '-g',
   200,
   '-m',
   5,
   '-d',
   '/opt/patric-common/data/kmer_metadata_v2',
   '-u',
   'http://pear.mcs.anl.gov:6100/query'],
  'hostname': 'pear'},
 {'tool_name': 'KmerAnnotationByFigfam',
  'parameters': ['annotate_hypothetical_only=1',
   'dataset_name=Release70',
   'kmer_size=8'],
  'hostname': 'pear',
  'id': '0F1D4994-CE8A-11EB-B500-6F3DBCF382BD',
  'execution_time': 1623837791.93171},
 {'id': '0F5608E2-CE8A-11EB-8811-45B0BCF382BD',
  'tool_name': 'annotate_proteins_similarity',
  'hostname': 'pear',
  'execute_time': 1623837792.30363,
  'parameters': []}]

In [14]:
model_kbase.info.type

'KBaseFBA.FBAModel'

In [8]:
model_kbase_json = kbase.get_object('ecoli_model', ws)

In [13]:
model_kbase = kbase.get_from_ws('ecoli_model', ws)

In [10]:
to_json = model_kbase._to_json()

In [11]:
to_json == model_kbase_json

True

In [18]:
model_kbase_json['modelcompartments']

[{'compartmentIndex': 0,
  'compartment_ref': '~/template/compartments/id/c',
  'id': 'c0',
  'label': 'Cytosol_0',
  'pH': 7,
  'potential': 0},
 {'compartmentIndex': 0,
  'compartment_ref': '~/template/compartments/id/e',
  'id': 'e0',
  'label': 'Extracellular_0',
  'pH': 7,
  'potential': 0}]

In [17]:
model_modelseed_json['modelcompartments']

[{'compartmentIndex': 0,
  'id': 'c0',
  'label': '',
  'compartment_ref': '~/template/compartments/id/c',
  'pH': 7,
  'potential': 0},
 {'compartmentIndex': 0,
  'id': 'e0',
  'label': '',
  'compartment_ref': '~/template/compartments/id/c',
  'pH': 7,
  'potential': 0}]

In [ ]:
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
model_modelseed_json, model_modelseed_fbamodel = CobraModelConverter(model_modelseed, genome, template).build()

In [18]:
kbase.save_object('ecoli_modelseedpy', 'filipeliu:narrative_1623733614496', model_modelseed_fbamodel.info.type, model_modelseed_json)

[[11,
  'ecoli_modelseedpy',
  'KBaseFBA.FBAModel-12.0',
  '2021-06-16T08:30:30+0000',
  3,
  'filipeliu',
  92991,
  'filipeliu:narrative_1623733614496',
  'e96e8d34e1deba910dac1e2aa4e99976',
  1567072,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'COBRA KBase',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Source': 'COBRA KBase',
   'Number compounds': '1446',
   'Number reactions': '1560',
   'Name': 'ecoli'}]]

In [105]:
for r in model_modelseed.reactions:
    if r.id == 'rxn00646_c0':
        proteins = []
        model_reaction_data = {
                'id': r.id,
                'name': r.name,
                'direction': '>',
                'aliases': [],
                'coverage': 3,
                'dblinks': {},
                'edits': {}, 'gapfill_data': {},
                'gene_count': 3,
                'maxforflux': 1000000,
                'maxrevflux': 1000000,
                'modelReactionProteins': proteins,
                'modelReactionReagents': [],
                'modelcompartment_ref': '~/modelcompartments/id/c0',
                'numerical_attributes': {}, 'string_attributes': {},
                'probability': 1,
                'protons': 0,
                'reaction_ref': '~/template/reactions/id/' + r.id[:-1],
            }
        reaction = ModelReaction(model_reaction_data)
        reaction.add_metabolites(r.metabolites)
        print(r)
        print(reaction)
        break

rxn00646_c0: cpd00002_c0 + cpd00023_c0 + cpd00084_c0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd00506_c0
rxn00646_c0: cpd00002_c0 + cpd00023_c0 + cpd00084_c0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd00506_c0


In [18]:
r

Reaction identifier,SK_cpd15302_c0
Name,Sink for glycogen(n-1)
Memory address,0x07f9cadb25410
Stoichiometry,cpd15302_c0 --> glycogen(n-1) -->
GPR,
Lower bound,0
Upper bound,1000


In [24]:
model_modelseed.compartments

{'c0': '', 'e0': ''}

In [120]:
def tt_to_json(r):
    rev, max_flux, min_flux = r._get_rev_and_max_min_flux()
    model_reaction_reagents = []
    model_reaction_proteins = []
    s = r.metabolites
    for m in s:
        model_reaction_reagents.append({
            'coefficient': s[m],
            'modelcompound_ref': '~/modelcompounds/id/' + m.id
        })
    return {
        'id': r.id,
        'name': r.name,
        'direction': rev,
        'maxforflux': max_flux,
        'maxrevflux': min_flux,
        'aliases': [],
        'dblinks': {},
        'edits': {},
        'gapfill_data': {},
        'modelReactionProteins': model_reaction_proteins,
        'modelReactionReagents': model_reaction_reagents,
        'modelcompartment_ref': '~/modelcompartments/id/' + r.compartment,
        'numerical_attributes': {},
        'probability': 0,
        'protons': 0,
        'reaction_ref': '~/template/reactions/id/' + r.id[:-1],
        'string_attributes': {}
    }
tt_to_json(reaction)

{'id': 'rxn00646_c0',
 'name': 'L-glutamate:L-cysteine gamma-ligase (ADP-forming)_c0',
 'direction': '=',
 'maxforflux': 1000,
 'maxrevflux': 0.0,
 'aliases': [],
 'dblinks': {},
 'edits': {},
 'gapfill_data': {},
 'modelReactionProteins': [],
 'modelReactionReagents': [{'coefficient': -1,
   'modelcompound_ref': '~/modelcompounds/id/cpd00023_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00084_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00002_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00008_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00009_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00506_c0'}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn00646_c',
 'stri

In [61]:
debug['template_refs']

['']

In [70]:
debug['modelcompartments']

[{'compartmentIndex': 0,
  'compartment_ref': '~/template/compartments/id/c',
  'id': 'c0',
  'label': '',
  'pH': 7,
  'potential': 0}]

In [20]:
import cobra.test
model = cobra.test.create_test_model("textbook")

In [23]:
r = model.reactions.get_by_id('ICDHyr')

In [26]:
r.gene_reaction_rule

'b1136'

In [13]:
debug = kbase.get_object('ecoli_modelseedpy_direct', ws)

In [16]:
for k in debug:
    v = debug[k]
    if type(v) == list:
        print(k, '\tList:', len(v))
    else:
        print(k,'\t[', v, ']')

__VERSION__ 	[ 1 ]
attributes 	[ {'auxotrophy': {}, 'fbas': {}, 'gene_count': 0, 'pathways': {}} ]
biomasses 	List: 1
contig_coverages 	[ {} ]
delete_biomasses 	[ {} ]
deleted_reactions 	[ {} ]
gapfilledcandidates 	List: 0
gapfillings 	List: 0
gapgens 	List: 0
id 	[ ecoli_modelseedpy_direct ]
loops 	List: 0
model_edits 	List: 0
modelcompartments 	List: 2
modelcompounds 	List: 1446
modelreactions 	List: 1560
name 	[ ecoli ]
other_genome_refs 	List: 0
quantopts 	List: 0
source 	[ ModelSEEDpy ]
source_id 	[ ecoli ]
template_ref 	[ 12998/10/6 ]
template_refs 	List: 1
type 	[ GenomeScale ]


In [15]:
debug['template_ref'] = str(template.info)
debug['template_refs'] = [str(template.info)]

In [17]:
kbase.save_object('ecoli_modelseedpy_direct', ws, model_kbase.info.type, debug)

[[17,
  'ecoli_modelseedpy_direct',
  'KBaseFBA.FBAModel-12.0',
  '2021-06-16T10:04:26+0000',
  8,
  'filipeliu',
  92991,
  'filipeliu:narrative_1623733614496',
  '237e105b19a24b2496f30213c8483664',
  1569437,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'ecoli',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Source': 'ModelSEEDpy',
   'Number compounds': '1446',
   'Number reactions': '1560',
   'Name': 'ecoli'}]]

In [97]:
reaction = model_modelseed.reactions[6]

In [ ]:
model = kbase.get_from_ws

In [23]:
from cobra.core import Model

In [8]:
model1 = kbase.get_object('ecoli_modelseedpy_direct', ws)
model2 = kbase.get_object('ecoli_kbase_classic', ws)

In [9]:
compounds1 = dict(map(lambda x: (x['id'], x), model1['modelcompounds']))
compounds2 = dict(map(lambda x: (x['id'], x), model2['modelcompounds']))
reactions1 = dict(map(lambda x: (x['id'], x), model1['modelreactions']))
reactions2 = dict(map(lambda x: (x['id'], x), model2['modelreactions']))

In [10]:
def compare_compound(m1, m2):
    assert m1['id'] == m2['id']
    #assert m1['name'] == m2['name']
    assert m1['compound_ref'] == m2['compound_ref']
    assert m1['modelcompartment_ref'] == m2['modelcompartment_ref']
    assert m1['formula'] == m2['formula']

In [11]:

for m1 in compounds1:
    if m1 in compounds2:
        compare_compound(compounds1[m1], compounds2[m1])
    else:
        print(m1)
for m2 in compounds2:
    if m2 not in compounds1:
        print(m2, compounds2[m2]['name'])

cpd00215_e0 Pyridoxal_e0
cpd00218_e0 Niacin_e0
cpd00220_e0 Riboflavin_e0
cpd00017_e0 S-Adenosyl-L-methionine_e0
cpd00393_e0 Folate_e0
cpd00084_e0 L-Cysteine_e0
cpd00104_e0 BIOT_e0


In [31]:
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
model_modelseed_json, model_modelseed_fbamodel = CobraModelConverter(model_modelseed, None, None).build()

In [35]:
model_modelseed_fbamodel.reactions.rxn08502_e0._to_json()

{'id': 'rxn08502_e0',
 'name': 'enterobactin Fe(III) binding (spontaneous)_c0',
 'direction': '=',
 'maxforflux': 1000,
 'maxrevflux': 0.0,
 'aliases': [],
 'dblinks': {},
 'edits': {},
 'gapfill_data': {},
 'modelReactionProteins': [],
 'modelReactionReagents': [{'modelcompound_ref': '~/modelcompounds/id/cpd03726_e0',
   'coefficient': 1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd03453_e0',
   'coefficient': -1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd10516_e0',
   'coefficient': -1.0}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn08502_e',
 'string_attributes': {}}

'k0'

In [37]:
r = model_modelseed.reactions.rxn13783_c0

In [38]:
r

Reaction identifier,rxn13783_c0
Name,DNA replication_c0
Memory address,0x07fb9fa7d9710
Stoichiometry,--> cpd17042_c0 --> DNA replication
GPR,(b4059) or (b0060) or (b2231) or (b3699) or (b1274) or (b0215) or (b4259) or (b3066) or (b3935) o...
Lower bound,0
Upper bound,1000


In [24]:
reactions2[r1]

{'aliases': [],
 'coverage': 0,
 'dblinks': {},
 'direction': '>',
 'edits': {},
 'gapfill_data': {},
 'gene_count': 0,
 'id': 'rxn08502_e0',
 'maxforflux': 1000000,
 'maxrevflux': 1000000,
 'modelReactionProteins': [{'complex_ref': '',
   'modelReactionProteinSubunits': [],
   'note': 'spontaneous',
   'source': ''}],
 'modelReactionReagents': [{'coefficient': -1,
   'modelcompound_ref': '~/modelcompounds/id/cpd03453_e0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd10516_e0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd03726_e0'}],
 'modelcompartment_ref': '~/modelcompartments/id/e0',
 'name': 'enterobactin Fe(III) binding (spontaneous)_e0',
 'numerical_attributes': {},
 'probability': 1,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn08502_e',
 'string_attributes': {}}

In [35]:
def assert_reaction(r1, r2):
    s1 = dict(map(lambda x: (x['modelcompound_ref'], x['coefficient']), r1['modelReactionReagents']))
    s2 = dict(map(lambda x: (x['modelcompound_ref'], x['coefficient']), r2['modelReactionReagents']))
    
    assert r1['id'] == r2['id']
    assert s1 == s2
    assert r1['direction'] == r2['direction']
    assert r1['reaction_ref'] == r2['reaction_ref']
    assert r1['modelcompartment_ref'] == r2['modelcompartment_ref']
    #print(r1['probability'])
    #print(r2['probability'])
    #print(r1['modelReactionProteins'])
    #print(r2['modelReactionProteins'])
    #print(r2)
    #print(r1)
    #print(r1['name'])
    #print(r2['name'])
    #assert r1['name'] == r2['name']
    

assert_reaction(reactions1[r1], reactions2[r1])

In [36]:
for r1 in reactions1:
    if r1 in reactions2:
        assert_reaction(reactions1[r1], reactions2[r1])
    else:
        print(r1, reactions1[r1]['name'])
for r2 in reactions2:
    if r2 not in reactions1:
        print(r2, reactions2[r2]['name'])

rxn03038_c0 R04376_c0
rxn04603_c0 R06788_c0
rxn13783_c0 DNA replication_c0
rxn13784_c0 RNA transcription_c0
rxn13782_c0 Protein biosynthesis_c0
rxn05299_c0 TRANS-RXNBWI-115637.ce.maizeexp.HIS_c0
rxn05244_c0 TRANS-RXNBWI-115637.ce.maizeexp.ILE_c0
rxn12666_c0 Pyridoxal transport_c0
rxn05308_c0 Pantothenate reversible transport via proton symport_c0
rxn05305_c0 L-lysine transport out via proton antiport reversible_c0
rxn09672_c0 TRANS-RXNBWI-115637.ce.maizeexp.MET_c0
rxn00068_c0 Fe2+:NAD+ oxidoreductase_c0
rxn05310_c0 Nicotinate transport (Plasma membrane)_c0
rxn05243_c0 TRANS-RXNBWI-115637.ce.maizeexp.LEU_c0
rxn05645_c0 riboflavin transport in via proton symport_c0
rxn09678_c0 TRANS-RXNBWI-115637.ce.maizeexp.GLN_c0
rxn09696_c0 S-adenosylmethionine permease SAM3_c0
rxn05255_c0 folate transport via proton simport_c0
rxn05303_c0 TRANS-RXNBWI-115637.ce.maizeexp.ARG_c0
rxn05669_c0 TRANS-RXNBWI-115637.ce.maizeexp.VAL_c0
rxn09657_c0 Thiamine transporter_c0
rxn09690_c0 TRANS-RXNBWI-115637.ce.mai

In [24]:
len(model1['modelreactions'])

1561

In [25]:
len(model2['modelreactions'])

1580

In [26]:
len(model1['modelcompounds'])

1448

In [27]:
len(model2['modelcompounds'])

1455

In [42]:
genome = kbase.get_from_ws('ecoli_genome.rast', ws)

In [43]:
ecoli_genome_rast_annotation = {}
for feature in genome.features:
    ecoli_genome_rast_annotation[feature.id] = list(feature.functions)
with open('ecoli_genome_rast_annotation.json', 'w') as fh:
    fh.write(json.dumps(ecoli_genome_rast_annotation))

In [39]:
import logging
logger = logging.getLogger(__name__)

In [42]:
logger.warning('! %s', 1)